In [138]:
import pymysql  
import nbimporter   
import TableCreation 

In [139]:
%run TableCreation.ipynb

Connected successfully!
Table 'patients_details' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('patient_changes',), ('patient_details',), ('patients',), ('treatment_details',), ('treatments',))
Table 'treatment' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('patient_changes',), ('patient_details',), ('patients',), ('treatment_details',), ('treatments',))
Table 'difficulty' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('patient_changes',), ('patient_details',), ('patients',), ('treatment_details',), ('treatments',))
Table 'patient_changes' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('patient_changes',), ('patient_details',), ('patients',), ('treatment_details',), ('treatments',))
Table 'pain_symptoms' created successfully!
Tables: (('difficulty_ratings',), ('medical_assistant_data',), ('pain_symptoms',), ('patient_changes',), (

In [140]:
import cv2
import numpy as np
import easyocr
import json

# Load the image
image_path = "/Users/akhilch/Desktop/example.png"  # Your image path
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Define bounding boxes (field_name: (x, y, width, height))
bounding_boxes = {
    "patient_name": (217, 191, 347, 58),
    "dob": (642, 185, 282, 72),
    "date": (128, 31, 225, 70),  
    "injection": (191, 111, 93, 35),
    "exercise_therapy": (849, 111, 91, 35),
    
    "Bending_or_stooping": (268, 297, 126, 52),
    "Putting_on_shoes":(232, 350, 118, 46),
    "Sleeping":(150, 402, 127, 47),
    "Standing_for_an_hour":(267, 451, 118, 55),
    "Going_up_or_down_a_flight_of_stairs":(403, 503, 115, 53),
    "Walking_through_a_store":(306, 557, 108, 39),
    "Driving_for_an_hour":(253, 609, 109, 36),
    "Preparing_a_meal":(231, 658, 125, 43),
    "Yard_work":(166, 707, 124, 43),
    "Picking_up_itens_off_the_floor": (351, 751, 122, 48),
    


    "since_last_treatment":(67, 846, 460, 69),
    "since_start_of_treatment":(68, 942, 463, 71),
    "last_3_days":(72, 1050, 472, 64),
    "pain":(116, 1150, 69, 49),
    "numbness":(299, 1147, 84, 47),
    "tingling":(465, 1150, 91, 42),
    "burning":(644, 1149, 81, 47),
    "tightness":(827, 1144, 63, 50),
    "Blood_Pressure":(215, 1245, 110, 46),
    "HR":(368, 1245, 91, 51),
    "Weight":(548, 1250, 100, 38),
    "Height":(735, 1243, 98, 44),
    "Program_Number":(247, 1299, 65, 37),
    "Treatment_Number":(513, 1299, 79, 35),
    "Placement":(703, 1296, 233, 42),
    "SpO2":(130, 1343, 78, 45),
    "Temperature":(347, 1340, 74, 48),
    "Blood_Glucose":(580, 1341, 76, 48),
    "Respirations":(786, 1346, 75, 48)
    
    

    
}

# Initialize OCR reader
reader = easyocr.Reader(['en'])

# Dictionary to store results
output_data = {
    "patient_name": "",
    "dob": "",
    "date": "",

    
    "injection": "",
    "exercise_therapy": "",
    "Bending_or_stooping": "",
    "Putting_on_shoes": "",
    "Sleeping":"",
    "Standing_for_an_hour":"",
    "Going_up_or_down_a_flight_of_stairs":"",
    "Walking_through_a_store":"",
    "Driving_for_an_hour":"",
    "Preparing_a_meal":"",
    "Yard_work":"",
    "Picking_up_itens_off_the_floor": "",
    




    
    "since_last_treatment": "",
    "since_start_of_treatment": "",
    
    "last_3_days":"",
    "pain":"",
    "numbness":"",
    "tingling":"",
    "burning":"",
    "tightness":"",
    "Blood_Pressure":"",
    "HR":"",
    "Weight":"",
    "Height":"",
    "Program_Number":"",
    "Treatment_Number":"",
    "Placement":"",
    "SpO2":"",
    "Temperature":"",
    "Blood_Glucose":"",
    "Respirations":""
    
}

# Extract text for simple fields
for field in ["patient_name", "dob", "date", "since_last_treatment", 
             "since_start_of_treatment", "last_3_days",
              
             "pain","numbness","tingling","burning","tightness","Blood_Pressure","HR","Weight","Height",
             "Program_Number","Treatment_Number","Placement","SpO2","Temperature","Blood_Glucose","Respirations"]:
    x, y, w, h = bounding_boxes[field]
    roi = gray[y:y+h, x:x+w]
    
    # Run OCR on the extracted region
    result = reader.readtext(roi, detail=0)
    output_data[field] = result[0] if result else ""

# Extract circled numbers
for field in ["injection", "exercise_therapy", "Bending_or_stooping", "Putting_on_shoes", "Sleeping", "Standing_for_an_hour",
             "Going_up_or_down_a_flight_of_stairs", "Walking_through_a_store", "Driving_for_an_hour", "Preparing_a_meal", "Yard_work", 
             "Picking_up_itens_off_the_floor"]:
    x, y, w, h = bounding_boxes[field]
    roi = gray[y:y+h, x:x+w]

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(roi, (9, 9), 2)

    # Detect circles inside the bounding box
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
                               param1=50, param2=30, minRadius=10, maxRadius=40)

    if circles is not None:
        circles = np.uint16(np.around(circles))  # Round values
        for (cx, cy, r) in circles[0, :]:
            # Adjust circle coordinates relative to full image
            absolute_x = x + cx
            absolute_y = y + cy

            # Extract the region of interest (ROI) around the detected circle
            circled_roi = gray[y+cy-r:y+cy+r, x+cx-r:x+cx+r]

            # Run OCR on the cropped circle area
            result = reader.readtext(circled_roi, detail=0)
            text = result[0] if result else "?"

            # Store the detected number
            output_data[field] = text

# Convert extracted data to JSON format
json_output = json.dumps(output_data, indent=4)

# Save JSON output to a file
with open("output.json", "w") as json_file:
    json_file.write(json_output)

json_output = json.loads(json_output)
# Print JSON output
print(json_output)



{'patient_name': 'AKhIL', 'dob': 'lo', 'date': 'Ie/o3/2025', 'injection': 'NO', 'exercise_therapy': 'YES', 'Bending_or_stooping': '16', 'Putting_on_shoes': '3', 'Sleeping': '?', 'Standing_for_an_hour': '?', 'Going_up_or_down_a_flight_of_stairs': '2)', 'Walking_through_a_store': '12', 'Driving_for_an_hour': '', 'Preparing_a_meal': '5', 'Yard_work': '8', 'Picking_up_itens_off_the_floor': '', 'since_last_treatment': '', 'since_start_of_treatment': '', 'last_3_days': '', 'pain': '', 'numbness': '', 'tingling': '', 'burning': '', 'tightness': '', 'Blood_Pressure': '', 'HR': '', 'Weight': '', 'Height': '', 'Program_Number': '', 'Treatment_Number': '', 'Placement': '', 'SpO2': '', 'Temperature': '', 'Blood_Glucose': '', 'Respirations': ''}


In [141]:
# Assuming json_output is a dictionary containing patient_name and dob
query = """INSERT INTO patient_details (patient_name, dob) 
           VALUES (%s, %s)"""

data = (json_output["patient_name"], json_output["dob"])

try:
    with connection.cursor() as cursor:
        cursor.execute(query, data)  # Safe execution
        patient_id = cursor.lastrowid
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")

✅ Data inserted successfully!


In [142]:
treatment_details = (patient_id,json_output["date"],json_output["injection"],json_output["exercise_therapy"])
treatment_query = """INSERT INTO treatment_details (patient_id, date,injection,exercise_therapy) 
           VALUES (%s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(treatment_query, treatment_details)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(14, 'Ie/o3/2025', 'NO', 'YES')


In [143]:
print(treatment_details)

(14, 'Ie/o3/2025', 'NO', 'YES')


In [144]:


difficulty_ratings = (patient_id ,json_output["date"],json_output["Bending_or_stooping"],json_output["Putting_on_shoes"],json_output["Sleeping"])


difficulty_query = """INSERT INTO difficulty_ratings (patient_id, date,bending_or_stooping,putting_on_shoes,sleeping) 
           VALUES (%s, %s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(difficulty_query,difficulty_ratings)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(14, 'Ie/o3/2025', 'NO', 'YES')


In [145]:
print(difficulty_ratings)

(14, 'Ie/o3/2025', '16', '3', '?')


In [146]:


patient_changes = (patient_id ,json_output["date"],json_output["since_last_treatment"],json_output["since_start_of_treatment"],json_output["last_3_days"])


patient_query = """INSERT INTO patient_changes(patient_id, date,since_last_treatment,since_start_of_treatment,last_3_days) 
           VALUES (%s, %s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(patient_query,patient_changes)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(14, 'Ie/o3/2025', 'NO', 'YES')


In [147]:


pain_symptoms= (patient_id ,json_output["date"],json_output["pain"],json_output["numbness"],json_output["tingling"],json_output["burning"],json_output["tightness"])


pain_query = """INSERT INTO pain_symptoms(patient_id, date,pain,numbness,tingling,burning,tightness) 
           VALUES (%s, %s, %s, %s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(pain_query,pain_symptoms)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)

✅ Data inserted successfully!
(14, 'Ie/o3/2025', 'NO', 'YES')


In [ ]:


medical_assistant_data= (patient_id ,json_output["date"],json_output["blood_pressure"],json_output["hr"],json_output["weight"],json_output["height"],json_output["spo2"],json_output["temperature"],json_output["blood_glucose"],json_output["respirations"])


pain_query = """INSERT INTO medical_assistant_data(patient_id, date,pain,numbness,tingling,burning,tightness) 
           VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""
try:
    with connection.cursor() as cursor:
        cursor.execute(pain_query,pain_symptoms)  # Safe execution
    connection.commit()  # Save changes
    print("✅ Data inserted successfully!")
except pymysql.MySQLError as e:
    print(f"❌ Error inserting data: {e}")
print(treatment_details)